In [1]:
from utils import *
from model3 import build_decoder_model

%load_ext autoreload
%autoreload 2

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vocab = Vocab("finished_files/vocab", VOCAB_SIZE)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [3]:
model = build_decoder_model(None)

In [4]:
#model = build_decoder_model(None)
model.load_model("full_retr2")
torch.cuda.memory_allocated()

271219712

In [5]:
class TestBatcher():
    
    def __init__(self, vocab, filename, max_article_len, max_target_len):
        generator = example_gen(filename)

        self.batches = []
        while True:
            
            try:
                article_text, abstract_text = next(generator)
                
                article_ids, unknown_words_list = article2ids(article_text, vocab)
                art_len = min(max_article_len, len(article_ids))
                article_ids = article_ids[:art_len]
                
                target = abstract2ids(abstract_text, vocab, unknown_words_list)
                tar_len = min(max_target_len, len(target))
                target.append(STOP_DECODING_ID)
                target = target[:tar_len]
                
                article_ids_unk = [i if i < VOCAB_SIZE else UNKNOWN_TOKEN_ID for i in article_ids]
                
                self.batches.append((np.array([article_ids]).T,np.array([article_ids_unk]).T, np.array(target), unknown_words_list))
                #print(article_text)
            except:
                break
        print(len(self.batches))
    
    def generator(self):
        for batch in self.batches:
            yield batch

In [6]:
test_data = TestBatcher(vocab, "finished_files/test.bin", 400, 80)

11490


In [7]:
import codecs
from beam_search import run_beam_search
import os

def test_generate(model, data, beam_width, mod=0):  
    model.train(False)
    sbleu = 0
    with torch.autograd.set_grad_enabled(False):
        for i, (article_text_full, article_text, target, unk_words) in enumerate(data.generator()):
            #result = generate_summary(article_text, model, beam_width)
            
            '''X_batch = LongTensor(article_text)
            inputs = InputLayerState("input", True, X_batch)
            batch_size = X_batch.shape[1]
            symbols = LongTensor([[START_DECODING_ID]*batch_size])
            result = []
            model.eval_run_encoder(inputs, beam_search=False)
            for k in range(100):
                hidden = model.decode(symbols)
                symbols = hidden.argmax(-1)
                result.append(symbols.squeeze(0).item())
            '''
            X_batch, X_batch_full = LongTensor(article_text), LongTensor(article_text_full)
            inputs = [InputLayerState("input", True, X_batch), InputLayerState("input_full", True, X_batch_full)]
            result = run_beam_search(model, vocab, inputs, beam_width, 80)
            #print(result)
            #print(target)
            
            target = clean_sentence(target)
            result = clean_sentence(result)
            bleu = sentence_bleu([result], target, weights=(0.5, 0.5, 0., 0.))
            result = outputids2words(result, vocab, unk_words)
            target = outputids2words(target, vocab, unk_words)
            #print('{:.4f}'.format(bleu))
            sbleu += bleu
            '''if (bleu < 1e-3):
                print(result)
                print(0)
                print(target)'''
            #print(result.split())
            #print('target is \n')
            #print(target.split())
            if not os.access('refs' + str(mod), os.F_OK):
                os.mkdir('refs' + str(mod), 777)
            if not os.access('res' + str(mod), os.F_OK):
                os.mkdir('res' + str(mod), 777)
            with codecs.open('refs' + str(mod) + '/ref.' + str(i) +'.txt', 'w', encoding='utf8') as reference_file:
                sentences = target.split('.')
                for s in sentences:
                    reference_file.write(s.strip() + '.\n')
                
                
            with codecs.open('res' + str(mod) + '/res.' + str(i) + '.txt', 'w', encoding='utf8') as result_file:
                sentences = result.split('.')
                for s in sentences:
                    result_file.write(s.strip() + '.\n')
        print(sbleu/1000)

In [8]:
test_generate(model, test_data, beam_width=4)

c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1.962560308457397


In [9]:
from pyrouge import Rouge155

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res0'
r.model_dir = 'E:/NIR/drag/refs0'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2019-06-15 13:59:58,554 [MainThread  ] [INFO ]  Writing summaries.
2019-06-15 13:59:58,556 [MainThread  ] [INFO ]  Processing summaries. Saving system files to C:\Users\Anatoly\AppData\Local\Temp\tmpouu3pb3d\system and model files to C:\Users\Anatoly\AppData\Local\Temp\tmpouu3pb3d\model.
2019-06-15 13:59:58,556 [MainThread  ] [INFO ]  Processing files in E:/NIR/drag/res0.
2019-06-15 13:59:58,575 [MainThread  ] [INFO ]  Processing res.0.txt.
2019-06-15 13:59:58,596 [MainThread  ] [INFO ]  Processing res.1.txt.
2019-06-15 13:59:58,603 [MainThread  ] [INFO ]  Processing res.10.txt.
2019-06-15 13:59:58,611 [MainThread  ] [INFO ]  Processing res.100.txt.
2019-06-15 13:59:58,625 [MainThread  ] [INFO ]  Processing res.1000.txt.
2019-06-15 13:59:58,643 [MainThread  ] [INFO ]  Processing res.10000.txt.
2019-06-15 13:59:58,651 [MainThread  ] [INFO ]  Processing res.10001.txt.
2019-06-15 13:59:58,658 [MainThread  ] [INFO ]  Processing res.10002.txt.
2019-06-15 13:59:58,664 [MainThread  ] [INFO ] 

2019-06-15 13:59:59,259 [MainThread  ] [INFO ]  Processing res.10092.txt.
2019-06-15 13:59:59,264 [MainThread  ] [INFO ]  Processing res.10093.txt.
2019-06-15 13:59:59,269 [MainThread  ] [INFO ]  Processing res.10094.txt.
2019-06-15 13:59:59,274 [MainThread  ] [INFO ]  Processing res.10095.txt.
2019-06-15 13:59:59,279 [MainThread  ] [INFO ]  Processing res.10096.txt.
2019-06-15 13:59:59,284 [MainThread  ] [INFO ]  Processing res.10097.txt.
2019-06-15 13:59:59,290 [MainThread  ] [INFO ]  Processing res.10098.txt.
2019-06-15 13:59:59,295 [MainThread  ] [INFO ]  Processing res.10099.txt.
2019-06-15 13:59:59,300 [MainThread  ] [INFO ]  Processing res.101.txt.
2019-06-15 13:59:59,308 [MainThread  ] [INFO ]  Processing res.1010.txt.
2019-06-15 13:59:59,315 [MainThread  ] [INFO ]  Processing res.10100.txt.
2019-06-15 13:59:59,320 [MainThread  ] [INFO ]  Processing res.10101.txt.
2019-06-15 13:59:59,325 [MainThread  ] [INFO ]  Processing res.10102.txt.
2019-06-15 13:59:59,330 [MainThread  ] [I

2019-06-15 13:59:59,877 [MainThread  ] [INFO ]  Processing res.10192.txt.
2019-06-15 13:59:59,882 [MainThread  ] [INFO ]  Processing res.10193.txt.
2019-06-15 13:59:59,888 [MainThread  ] [INFO ]  Processing res.10194.txt.
2019-06-15 13:59:59,893 [MainThread  ] [INFO ]  Processing res.10195.txt.
2019-06-15 13:59:59,898 [MainThread  ] [INFO ]  Processing res.10196.txt.
2019-06-15 13:59:59,903 [MainThread  ] [INFO ]  Processing res.10197.txt.
2019-06-15 13:59:59,909 [MainThread  ] [INFO ]  Processing res.10198.txt.
2019-06-15 13:59:59,914 [MainThread  ] [INFO ]  Processing res.10199.txt.
2019-06-15 13:59:59,919 [MainThread  ] [INFO ]  Processing res.102.txt.
2019-06-15 13:59:59,926 [MainThread  ] [INFO ]  Processing res.1020.txt.
2019-06-15 13:59:59,934 [MainThread  ] [INFO ]  Processing res.10200.txt.
2019-06-15 13:59:59,940 [MainThread  ] [INFO ]  Processing res.10201.txt.
2019-06-15 13:59:59,945 [MainThread  ] [INFO ]  Processing res.10202.txt.
2019-06-15 13:59:59,950 [MainThread  ] [I

2019-06-15 14:00:00,517 [MainThread  ] [INFO ]  Processing res.10292.txt.
2019-06-15 14:00:00,522 [MainThread  ] [INFO ]  Processing res.10293.txt.
2019-06-15 14:00:00,527 [MainThread  ] [INFO ]  Processing res.10294.txt.
2019-06-15 14:00:00,532 [MainThread  ] [INFO ]  Processing res.10295.txt.
2019-06-15 14:00:00,538 [MainThread  ] [INFO ]  Processing res.10296.txt.
2019-06-15 14:00:00,543 [MainThread  ] [INFO ]  Processing res.10297.txt.
2019-06-15 14:00:00,548 [MainThread  ] [INFO ]  Processing res.10298.txt.
2019-06-15 14:00:00,552 [MainThread  ] [INFO ]  Processing res.10299.txt.
2019-06-15 14:00:00,558 [MainThread  ] [INFO ]  Processing res.103.txt.
2019-06-15 14:00:00,567 [MainThread  ] [INFO ]  Processing res.1030.txt.
2019-06-15 14:00:00,574 [MainThread  ] [INFO ]  Processing res.10300.txt.
2019-06-15 14:00:00,579 [MainThread  ] [INFO ]  Processing res.10301.txt.
2019-06-15 14:00:00,584 [MainThread  ] [INFO ]  Processing res.10302.txt.
2019-06-15 14:00:00,590 [MainThread  ] [I

2019-06-15 14:00:01,130 [MainThread  ] [INFO ]  Processing res.10392.txt.
2019-06-15 14:00:01,136 [MainThread  ] [INFO ]  Processing res.10393.txt.
2019-06-15 14:00:01,141 [MainThread  ] [INFO ]  Processing res.10394.txt.
2019-06-15 14:00:01,146 [MainThread  ] [INFO ]  Processing res.10395.txt.
2019-06-15 14:00:01,151 [MainThread  ] [INFO ]  Processing res.10396.txt.
2019-06-15 14:00:01,156 [MainThread  ] [INFO ]  Processing res.10397.txt.
2019-06-15 14:00:01,162 [MainThread  ] [INFO ]  Processing res.10398.txt.
2019-06-15 14:00:01,167 [MainThread  ] [INFO ]  Processing res.10399.txt.
2019-06-15 14:00:01,172 [MainThread  ] [INFO ]  Processing res.104.txt.
2019-06-15 14:00:01,179 [MainThread  ] [INFO ]  Processing res.1040.txt.
2019-06-15 14:00:01,187 [MainThread  ] [INFO ]  Processing res.10400.txt.
2019-06-15 14:00:01,192 [MainThread  ] [INFO ]  Processing res.10401.txt.
2019-06-15 14:00:01,197 [MainThread  ] [INFO ]  Processing res.10402.txt.
2019-06-15 14:00:01,202 [MainThread  ] [I

...

2019-06-15 14:02:30,572 [MainThread  ] [INFO ]  Processing ref.969.txt.
2019-06-15 14:02:30,579 [MainThread  ] [INFO ]  Processing ref.9690.txt.
2019-06-15 14:02:30,584 [MainThread  ] [INFO ]  Processing ref.9691.txt.
2019-06-15 14:02:30,589 [MainThread  ] [INFO ]  Processing ref.9692.txt.
2019-06-15 14:02:30,595 [MainThread  ] [INFO ]  Processing ref.9693.txt.
2019-06-15 14:02:30,610 [MainThread  ] [INFO ]  Processing ref.9694.txt.
2019-06-15 14:02:30,615 [MainThread  ] [INFO ]  Processing ref.9695.txt.
2019-06-15 14:02:30,620 [MainThread  ] [INFO ]  Processing ref.9696.txt.
2019-06-15 14:02:30,627 [MainThread  ] [INFO ]  Processing ref.9697.txt.
2019-06-15 14:02:30,632 [MainThread  ] [INFO ]  Processing ref.9698.txt.
2019-06-15 14:02:30,638 [MainThread  ] [INFO ]  Processing ref.9699.txt.
2019-06-15 14:02:30,643 [MainThread  ] [INFO ]  Processing ref.97.txt.
2019-06-15 14:02:30,650 [MainThread  ] [INFO ]  Processing ref.970.txt.
2019-06-15 14:02:30,657 [MainThread  ] [INFO ]  Process

2019-06-15 14:02:31,209 [MainThread  ] [INFO ]  Processing ref.9791.txt.
2019-06-15 14:02:31,215 [MainThread  ] [INFO ]  Processing ref.9792.txt.
2019-06-15 14:02:31,220 [MainThread  ] [INFO ]  Processing ref.9793.txt.
2019-06-15 14:02:31,225 [MainThread  ] [INFO ]  Processing ref.9794.txt.
2019-06-15 14:02:31,230 [MainThread  ] [INFO ]  Processing ref.9795.txt.
2019-06-15 14:02:31,236 [MainThread  ] [INFO ]  Processing ref.9796.txt.
2019-06-15 14:02:31,241 [MainThread  ] [INFO ]  Processing ref.9797.txt.
2019-06-15 14:02:31,246 [MainThread  ] [INFO ]  Processing ref.9798.txt.
2019-06-15 14:02:31,251 [MainThread  ] [INFO ]  Processing ref.9799.txt.
2019-06-15 14:02:31,257 [MainThread  ] [INFO ]  Processing ref.98.txt.
2019-06-15 14:02:31,264 [MainThread  ] [INFO ]  Processing ref.980.txt.
2019-06-15 14:02:31,271 [MainThread  ] [INFO ]  Processing ref.9800.txt.
2019-06-15 14:02:31,276 [MainThread  ] [INFO ]  Processing ref.9801.txt.
2019-06-15 14:02:31,281 [MainThread  ] [INFO ]  Proces

2019-06-15 14:02:31,817 [MainThread  ] [INFO ]  Processing ref.9893.txt.
2019-06-15 14:02:31,822 [MainThread  ] [INFO ]  Processing ref.9894.txt.
2019-06-15 14:02:31,828 [MainThread  ] [INFO ]  Processing ref.9895.txt.
2019-06-15 14:02:31,833 [MainThread  ] [INFO ]  Processing ref.9896.txt.
2019-06-15 14:02:31,844 [MainThread  ] [INFO ]  Processing ref.9897.txt.
2019-06-15 14:02:31,849 [MainThread  ] [INFO ]  Processing ref.9898.txt.
2019-06-15 14:02:31,855 [MainThread  ] [INFO ]  Processing ref.9899.txt.
2019-06-15 14:02:31,860 [MainThread  ] [INFO ]  Processing ref.99.txt.
2019-06-15 14:02:31,868 [MainThread  ] [INFO ]  Processing ref.990.txt.
2019-06-15 14:02:31,875 [MainThread  ] [INFO ]  Processing ref.9900.txt.
2019-06-15 14:02:31,880 [MainThread  ] [INFO ]  Processing ref.9901.txt.
2019-06-15 14:02:31,885 [MainThread  ] [INFO ]  Processing ref.9902.txt.
2019-06-15 14:02:31,890 [MainThread  ] [INFO ]  Processing ref.9903.txt.
2019-06-15 14:02:31,896 [MainThread  ] [INFO ]  Proces

2019-06-15 14:02:32,438 [MainThread  ] [INFO ]  Processing ref.9995.txt.
2019-06-15 14:02:32,444 [MainThread  ] [INFO ]  Processing ref.9996.txt.
2019-06-15 14:02:32,449 [MainThread  ] [INFO ]  Processing ref.9997.txt.
2019-06-15 14:02:32,454 [MainThread  ] [INFO ]  Processing ref.9998.txt.
2019-06-15 14:02:32,459 [MainThread  ] [INFO ]  Processing ref.9999.txt.
2019-06-15 14:02:32,465 [MainThread  ] [INFO ]  Saved processed files to C:\Users\Anatoly\AppData\Local\Temp\tmpouu3pb3d\model.
2019-06-15 14:06:46,319 [MainThread  ] [INFO ]  Written ROUGE configuration to C:\Users\Anatoly\AppData\Local\Temp\tmpzxovfv4j\rouge_conf.xml
2019-06-15 14:06:46,319 [MainThread  ] [INFO ]  Running ROUGE with command perl E:\NIR\pyrouge\tools\ROUGE-1.5.5\ROUGE-1.5.5.pl -e E:\NIR\pyrouge\tools\ROUGE-1.5.5\data -c 95 -2 -1 -U -r 1000 -n 4 -w 1.2 -a -m C:\Users\Anatoly\AppData\Local\Temp\tmpzxovfv4j\rouge_conf.xml


---------------------------------------------
1 ROUGE-1 Average_R: 0.44027 (95%-conf.int. 0.43791 - 0.44269)
1 ROUGE-1 Average_P: 0.29702 (95%-conf.int. 0.29488 - 0.29904)
1 ROUGE-1 Average_F: 0.34743 (95%-conf.int. 0.34530 - 0.34945)
---------------------------------------------
1 ROUGE-2 Average_R: 0.17623 (95%-conf.int. 0.17404 - 0.17849)
1 ROUGE-2 Average_P: 0.11874 (95%-conf.int. 0.11709 - 0.12039)
1 ROUGE-2 Average_F: 0.13882 (95%-conf.int. 0.13709 - 0.14066)
---------------------------------------------
1 ROUGE-3 Average_R: 0.09787 (95%-conf.int. 0.09589 - 0.09990)
1 ROUGE-3 Average_P: 0.06644 (95%-conf.int. 0.06502 - 0.06798)
1 ROUGE-3 Average_F: 0.07736 (95%-conf.int. 0.07576 - 0.07906)
---------------------------------------------
1 ROUGE-4 Average_R: 0.06362 (95%-conf.int. 0.06195 - 0.06543)
1 ROUGE-4 Average_P: 0.04347 (95%-conf.int. 0.04224 - 0.04481)
1 ROUGE-4 Average_F: 0.05043 (95%-conf.int. 0.04909 - 0.05195)
---------------------------------------------
1 ROUGE-L Aver

model.load_model("cov_loss2")

test_generate(model,test_data, beam_width=4, mod=1)

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res1'
r.model_dir = 'E:/NIR/drag/refs1'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.A.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

In [10]:
0.10897

0.10897